# **import編**

In [1]:
import torch
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import time

# !pip install deap
import deap

def load_cifar10(batch=128):
    train_loader = DataLoader(
        datasets.CIFAR10('./data',
                         train=True,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(
                                [0.5, 0.5, 0.5],  # RGB 平均
                                [0.5, 0.5, 0.5]   # RGB 標準偏差
                                )
                         ])),
        batch_size=batch,
        shuffle=True
    )

    test_loader = DataLoader(
        datasets.CIFAR10('./data',
                         train=False,
                         download=True,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(
                                 [0.5, 0.5, 0.5],  # RGB 平均
                                 [0.5, 0.5, 0.5]  # RGB 標準偏差
                             )
                         ])),
        batch_size=batch,
    )

    return {'train': train_loader, 'test': test_loader}

torch.cuda.is_available()

False

# **教師モデル定義(温度付き・dropout削除)**

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class VGG16(nn.Module):
    def __init__(self): # , num_classes):
        super(VGG16, self).__init__()
        num_classes=10

        self.block1_output = nn.Sequential (
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block2_output = nn.Sequential (
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block3_output = nn.Sequential (
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block4_output = nn.Sequential (
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block5_output = nn.Sequential (
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(512, 512),  #512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 32 ),  #4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(32, num_classes),  #4096
        )

    def forward(self, x,T):
        x = self.block1_output(x)
        # print(x.size())
        x = self.block2_output(x)
        # print(x.size())
        x = self.block3_output(x)
        # print(x.size())
        x = self.block4_output(x)
        # print(x.size())
        x = self.block5_output(x)
        # print(x.size())
        x = x.view(x.size(0), -1)
        # print(x.size())
        x = self.classifier(x)
        x = x/T
#         return f.softmax(x,dim=1)
        return x

# **VGG生成モデル②**

入力[n_1, n_2, , , ,  n_N]

VGG16(正確にはconvが13層)を生成しモデルが作られる。

In [3]:
class CreateVGG_2(torch.nn.Module):
    def __init__(self, VGG_model_list):
        super(CreateVGG_2, self).__init__()
        self.count = 0

        PRE_FILTER = 3
        for i in range(len(VGG_model_list)):
            if VGG_model_list[i]!=0:
                FILTER = VGG_model_list[i]
                self._modules["fc"+str(i+1)] = nn.Sequential (
                nn.Conv2d(PRE_FILTER, FILTER, kernel_size=3, padding=1),
                nn.BatchNorm2d(FILTER),
                nn.ReLU(inplace=True),
                )
                PRE_FILTER=FILTER

        self.classifier = nn.Sequential(
            nn.Linear(PRE_FILTER, PRE_FILTER), 
            nn.ReLU(True),
            nn.Linear(PRE_FILTER, PRE_FILTER//2 ), 
            nn.ReLU(True),
            nn.Linear(PRE_FILTER//2, 10), 
        )
        self.pool = torch.nn.MaxPool2d(2, 2) 
        

    def forward(self, x, T):
        for i in range(1,14):
            if "fc"+str(i) in self._modules:
                x = f.relu(self._modules["fc"+str(i)](x))
                # print(x.size())
            if i==2 or i==4 or i==7 or i==10 or i==13:
                x = self.pool(x)
                # print(x.size())
        x = x.view(x.size(0), -1)
        # print(x.size())
        x = self.classifier(x)
        # print(x.size())
        x = x/T
        return x

# **温度付きcross_entropyの損失関数の定義**

In [4]:
class T_S_Closs_entropy(torch.nn.Module):
    def __init__(self):
        super(T_S_Closs_entropy, self).__init__()

    def forward(self, p, q):
        #pが教師モデル、qが生徒モデルの出力値(既に対数化してます)。
        loss = torch.mul(p,q)
        loss = -torch.sum(loss)
        return loss

# **速度と精度の評価関数(最大化予定)**

In [5]:
def speed_acc_normalization(spe_max,spe_min,spe,acc_max,acc_min,acc):
    if spe_min<=spe<=spe_max:
        speed_normalization = ((spe_max-spe+1) - spe_min) / ( (spe_max - spe_min) * 2 )
    else:
        speed_normalization=0
    if acc_min<=acc<=acc_max:
        accuracy_normalization = (acc - acc_min) / ( (acc_max - acc_min) * 2 )
    else:
        accuracy_normalization=0
    print("--------------------------------------------------------------------------------")
    print("速度:{} ,精度:{}".format(spe,acc))
    print("速度評価値:{} ,精度評価値:{}".format(speed_normalization,accuracy_normalization))
    print("評価指標:{}".format(speed_normalization + accuracy_normalization))
    print("--------------------------------------------------------------------------------")

    return speed_normalization + accuracy_normalization

# **教師モデルのCPU評価確認**

In [6]:
Tnet: VGG16 = VGG16()
loader = load_cifar10()
device = torch.device('cpu')
Tnet.to(device)

param = torch.load('CPU_VGG16_weight_200ep.pth')
Tnet.load_state_dict(param)

Tnet_speed_list = []
Tnet_accuracy_list = []

Tnet_speed = 0
correct = 0
Tnet_accuracy = 0
for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = Tnet(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    Tnet_speed_list.append(t2-t1)
    Tnet_accuracy_list.append(correct/10000)

Tnet_speed = sum(Tnet_speed_list)/len(Tnet_speed_list)
Tnet_accuracy = sum(Tnet_accuracy_list)/len(Tnet_accuracy_list)
print(Tnet_speed_list)
print(Tnet_accuracy_list)

print(Tnet_speed)
print(Tnet_accuracy)

Files already downloaded and verified
Files already downloaded and verified
[111.84997081756592, 105.73307180404663, 105.57797980308533, 105.57011795043945, 105.48853397369385, 107.28955411911011, 105.55613374710083, 105.52286005020142, 105.19754385948181, 105.14894700050354, 105.13139915466309, 105.15191102027893, 105.13765788078308, 105.20713400840759, 105.52604389190674, 104.92085599899292, 105.34269094467163, 105.05850100517273, 105.00808072090149, 104.92165613174438]
[0.8628, 0.8622, 0.8633, 0.8618, 0.863, 0.861, 0.8595, 0.8622, 0.8623, 0.8613, 0.8631, 0.8607, 0.8602, 0.8626, 0.8615, 0.8602, 0.8635, 0.8636, 0.8634, 0.862]
105.71703219413757
0.8620099999999999


# **最適モデル①0.5-0.5**
## [2, 2, 3, 2, 0, 0, 4, 0, 3, 5, 3, 0, 0]

In [7]:
model_gane=[]
gane_number=[0,16,32,64,128,256,512,1024]
gane_in = [2, 2, 3, 2, 0, 0, 4, 0, 3, 5, 3, 0, 0]
for i in gane_in:
    if gane_number[i]!=0:
        model_gane.append(gane_number[i])

net: torch.nn.Module = CreateVGG_2(model_gane)
device = torch.device('cpu')
net.to(device)

param = torch.load('CPU_VGG_最適モデル_soft_05-05_2.pth')
net.load_state_dict(param)
loader = load_cifar10()


saiteki_1_speed_list = []
saiteki_1_accuracy_list = []

saiteki_1_speed = 0
correct = 0
saiteki_1_accuracy = 0
for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = net(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    saiteki_1_speed_list.append(t2-t1)
    saiteki_1_accuracy_list.append(correct/10000)

saiteki_1_speed = sum(saiteki_1_speed_list)/len(saiteki_1_speed_list)
saiteki_1_accuracy = sum(saiteki_1_accuracy_list)/len(saiteki_1_accuracy_list)
print(saiteki_1_speed_list)
print(saiteki_1_accuracy_list)
print(speed_acc_normalization(Tnet_speed,0,saiteki_1_speed,Tnet_accuracy,0,saiteki_1_accuracy))

Files already downloaded and verified
Files already downloaded and verified
[23.20385503768921, 22.28924012184143, 22.066208839416504, 22.153620958328247, 22.06828498840332, 22.15663194656372, 22.07717490196228, 22.01635503768921, 22.041237831115723, 22.158340215682983, 22.052953720092773, 22.791910886764526, 22.322813987731934, 22.457043886184692, 22.076854705810547, 22.112611055374146, 22.112446308135986, 22.054327964782715, 22.034154891967773, 22.129614114761353]
[0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508, 0.8508]
--------------------------------------------------------------------------------
速度:22.218784070014955 ,精度:0.8507999999999998
速度評価値:0.39964349343893324 ,精度評価値:0.4934977552464588
評価指標:0.893141248685392
--------------------------------------------------------------------------------
0.893141248685392


# **最適モデル①0.75-0.25**
## [3, 1, 1, 1, 4, 2, 0, 0, 0, 2, 3, 0, 3]

In [8]:
model_gane=[]
gane_number=[0,16,32,64,128,256,512,1024]
gane_in = [3, 1, 1, 1, 4, 2, 0, 0, 0, 2, 3, 0, 3]
for i in gane_in:
    if gane_number[i]!=0:
        model_gane.append(gane_number[i])

net: torch.nn.Module = CreateVGG_2(model_gane)
device = torch.device('cpu')
net.to(device)

param = torch.load('CPU_VGG_最適モデル_soft_075-025_2.pth')
net.load_state_dict(param)
loader = load_cifar10()


saiteki_2_speed_list = []
saiteki_2_accuracy_list = []

saiteki_2_speed = 0
correct = 0
saiteki_2_accuracy = 0
for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = net(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    saiteki_2_speed_list.append(t2-t1)
    saiteki_2_accuracy_list.append(correct/10000)

saiteki_2_speed = sum(saiteki_2_speed_list)/len(saiteki_2_speed_list)
saiteki_2_accuracy = sum(saiteki_2_accuracy_list)/len(saiteki_2_accuracy_list)
print(saiteki_2_speed_list)
print(saiteki_2_accuracy_list)
print(speed_acc_normalization(Tnet_speed,0,saiteki_2_speed,Tnet_accuracy,0,saiteki_2_accuracy))

Files already downloaded and verified
Files already downloaded and verified
[17.583630084991455, 17.501683950424194, 17.806393146514893, 19.621979236602783, 19.22533106803894, 19.08629584312439, 19.4196879863739, 19.536909103393555, 19.45315909385681, 18.25260019302368, 17.552335739135742, 17.586161851882935, 17.517055988311768, 17.547163009643555, 17.513054132461548, 17.90844488143921, 17.59416699409485, 17.536367177963257, 17.52637267112732, 17.541129112243652]
[0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049, 0.8049]
--------------------------------------------------------------------------------
速度:18.16549606323242 ,精度:0.8049
速度評価値:0.41881395217513345 ,精度評価値:0.4668739341770977
評価指標:0.8856878863522312
--------------------------------------------------------------------------------
0.8856878863522312


# **最適モデル③0.25-0.75**
## [0, 2, 3, 5, 5, 5, 6, 4, 3, 0, 6, 1, 6]

In [10]:
model_gane=[]
gane_number=[0,16,32,64,128,256,512,1024]
gane_in = [3, 2, 2, 3, 0, 5, 0, 3, 3, 0, 0, 3, 6]
for i in gane_in:
    if gane_number[i]!=0:
        model_gane.append(gane_number[i])

net: torch.nn.Module = CreateVGG_2(model_gane)
device = torch.device('cpu')
net.to(device)

param = torch.load('CPU_VGG_最適モデル_soft_025-075_4.pth')
net.load_state_dict(param)
loader = load_cifar10()



saiteki_3_speed_list = []
saiteki_3_accuracy_list = []

saiteki_3_speed = 0
correct = 0
saiteki_3_accuracy = 0


for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = net(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    saiteki_3_speed_list.append(t2-t1)
    saiteki_3_accuracy_list.append(correct/10000)

saiteki_3_speed = sum(saiteki_3_speed_list)/len(saiteki_3_speed_list)
saiteki_3_accuracy = sum(saiteki_3_accuracy_list)/len(saiteki_3_accuracy_list)
print(saiteki_3_speed_list)
print(saiteki_3_accuracy_list)
print(speed_acc_normalization(Tnet_speed,0,saiteki_3_speed,Tnet_accuracy,0,saiteki_3_accuracy))

Files already downloaded and verified
Files already downloaded and verified
[34.5208580493927, 33.71959590911865, 33.73486804962158, 34.48434114456177, 34.11375617980957, 33.50921702384949, 33.503000020980835, 33.479912996292114, 33.56878995895386, 33.375592947006226, 33.140745878219604, 33.40308403968811, 33.35543513298035, 33.10749101638794, 33.29514408111572, 33.31159806251526, 34.266167879104614, 33.261006116867065, 33.20918297767639, 33.10050082206726]
[0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582, 0.8582]
--------------------------------------------------------------------------------
速度:33.57301441431046 ,精度:0.8581999999999999
速度評価値:0.34594244778602123 ,精度評価値:0.4977900488393406
評価指標:0.8437324966253619
--------------------------------------------------------------------------------
0.8437324966253619


# **フィルム数1/2モデル**
## [2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5 ,5, 5]

In [11]:
model_gane=[]
gane_number=[0,16,32,64,128,256,512,1024]
gane_in = [2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5 ,5, 5]
for i in gane_in:
    if gane_number[i]!=0:
        model_gane.append(gane_number[i])

net: torch.nn.Module = CreateVGG_2(model_gane)
device = torch.device('cpu')
net.to(device)

param = torch.load('CPU_半フィルモデル.pth')
net.load_state_dict(param)
loader = load_cifar10()



han_speed_list = []
han_accuracy_list = []

han_speed = 0
correct = 0
han_accuracy = 0
for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = net(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    han_speed_list.append(t2-t1)
    han_accuracy_list.append(correct/10000)

han_speed = sum(han_speed_list)/len(han_speed_list)
han_accuracy = sum(han_accuracy_list)/len(han_accuracy_list)
print(han_speed_list)
print(han_accuracy_list)
print(speed_acc_normalization(Tnet_speed,0,han_speed,Tnet_accuracy,0,han_accuracy))

Files already downloaded and verified
Files already downloaded and verified
[37.36721920967102, 37.424785137176514, 37.655044078826904, 37.75069189071655, 37.480549812316895, 37.44814491271973, 37.444176197052, 37.42633581161499, 37.41899919509888, 37.40064811706543, 37.44123911857605, 37.53127694129944, 37.46248006820679, 37.484981060028076, 37.418144941329956, 37.39890789985657, 37.483752965927124, 37.45727300643921, 37.43508219718933, 37.49504208564758]
[0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573, 0.8573]
--------------------------------------------------------------------------------
速度:37.47123873233795 ,精度:0.8573000000000002
速度評価値:0.32750537933489005 ,精度評価値:0.4972680131320984
評価指標:0.8247733924669884
--------------------------------------------------------------------------------
0.8247733924669884


# **ランダムモデル①**
## [5, 2, 2, 3, 4, 2, 6, 0, 6, 1, 3, 1, 1]

In [12]:
model_gane=[]
gane_number=[0,16,32,64,128,256,512,1024]
gane_in = [5, 2, 2, 3, 4, 2, 6, 0, 6, 1, 3, 1, 1]
for i in gane_in:
    if gane_number[i]!=0:
        model_gane.append(gane_number[i])

net: torch.nn.Module = CreateVGG_2(model_gane)
device = torch.device('cpu')
net.to(device)

param = torch.load('CPU_ランダムモデル①.pth')
net.load_state_dict(param)
loader = load_cifar10()


randam_1_speed_list = []
randam_1_accuracy_list = []

randam_1_speed = 0
correct = 0
randam_1_accuracy = 0
for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = net(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    randam_1_speed_list.append(t2-t1)
    randam_1_accuracy_list.append(correct/10000)

randam_1_speed = sum(randam_1_speed_list)/len(randam_1_speed_list)
randam_1_accuracy = sum(randam_1_accuracy_list)/len(randam_1_accuracy_list)
print(randam_1_speed_list)
print(randam_1_accuracy_list)
print(speed_acc_normalization(Tnet_speed,0,randam_1_speed,Tnet_accuracy,0,randam_1_accuracy))

Files already downloaded and verified
Files already downloaded and verified
[73.63148903846741, 73.20057201385498, 73.29473996162415, 73.22236204147339, 73.2164158821106, 73.2011570930481, 78.20410919189453, 74.1192467212677, 74.11816906929016, 73.96177315711975, 74.20804405212402, 73.84660196304321, 74.07331490516663, 73.84855890274048, 75.28785586357117, 81.56861019134521, 76.12619924545288, 73.4449028968811, 73.51941990852356, 73.4626030921936]
[0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494, 0.8494]
--------------------------------------------------------------------------------
速度:74.47780725955963 ,精度:0.8493999999999996
速度評価値:0.1524788592030005 ,精度評価値:0.4926856997018594
評価指標:0.6451645589048599
--------------------------------------------------------------------------------
0.6451645589048599


# **ランダムモデル②**
## [3, 2, 5, 1, 3, 1, 3, 6, 3, 2, 3, 5, 3]

In [13]:
model_gane=[]
gane_number=[0,16,32,64,128,256,512,1024]
gane_in = [3, 2, 5, 1, 3, 1, 3, 6, 3, 2, 3, 5, 3]
for i in gane_in:
    if gane_number[i]!=0:
        model_gane.append(gane_number[i])

net: torch.nn.Module = CreateVGG_2(model_gane)
device = torch.device('cpu')
net.to(device)

param = torch.load('CPU_ランダムモデル②.pth')
net.load_state_dict(param)
loader = load_cifar10()


randam_2_speed_list = []
randam_2_accuracy_list = []

randam_2_speed = 0
correct = 0
randam_2_accuracy = 0
for i in range(20):
    correct = 0
    t1 = time.time()
    with torch.no_grad():
        for images, labels in loader['test']:
            images = images.to(device) 
            labels = labels.to(device)
            outputs = net(images,1)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    t2 = time.time()

    randam_2_speed_list.append(t2-t1)
    randam_2_accuracy_list.append(correct/10000)

randam_2_speed = sum(randam_2_speed_list)/len(randam_2_speed_list)
randam_2_accuracy = sum(randam_2_accuracy_list)/len(randam_2_accuracy_list)
print(randam_2_speed_list)
print(randam_2_accuracy_list)
print(speed_acc_normalization(Tnet_speed,0,randam_2_speed,Tnet_accuracy,0,randam_2_accuracy))

Files already downloaded and verified
Files already downloaded and verified
[34.6356258392334, 34.393192291259766, 34.350605964660645, 35.0462589263916, 37.136292934417725, 36.746511936187744, 38.60405921936035, 36.61305499076843, 34.394871950149536, 35.29369616508484, 35.781800985336304, 34.36730909347534, 34.35951375961304, 34.51644492149353, 34.63635802268982, 34.448204040527344, 34.47697114944458, 34.451547145843506, 34.47940921783447, 34.427387952804565]
[0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305, 0.8305]
--------------------------------------------------------------------------------
速度:35.15795582532883 ,精度:0.8305000000000003
速度評価値:0.3384462980260289 ,精度評価値:0.48172294984976993
評価指標:0.8201692478757988
--------------------------------------------------------------------------------
0.8201692478757988


---
---
---
---
---
---